In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [4]:
from pyspark.sql.functions import to_timestamp, col, concat, lit
from pyspark.sql.types import *

In [95]:
# VietstockFinance_companies tranform
from pyspark.sql.functions import expr

copanies_with_uuid = companies\
    .withColumnRenamed("Sàn CK","stock_exchange")\
    .withColumnRenamed("Mã CK","stock_code")\
    .withColumnRenamed("Tên công ty","copany_name")\
    .distinct()\
    .withColumn("stock_id", expr("uuid()"))
copanies_with_uuid.write.format("json").mode("overwrite").save("hdfs://namenode:9000/project20221/clean/VietstockFinance_companies.json")
# companies_clean = spark.read.json("hdfs://namenode:9000/project20221/clean/VietstockFinance_companies.json")
# companies_clean.show()

In [5]:
from datetime import datetime, timedelta

time_stamp = '01-10-2023-21-45-01';
time_stamp = datetime.strptime(time_stamp, '%m-%d-%Y-%H-%M-%S')

companies = spark.read.json("hdfs://namenode:9000/project20221/raw/stock_price_realtime/01-11-2023-22-47-37.json")
        
stock_info_clean = companies\
    .distinct()\
    .withColumn('best1Bid', col('best1Bid') /1000)\
    .withColumn('best2Bid', col('best2Bid') /1000)\
    .withColumn('best3Bid', col('best3Bid') /1000)\
    .withColumn('best1Offer', col('best1Offer') /1000)\
    .withColumn('best2Offer', col('best2Offer') /1000)\
    .withColumn('best3Offer', col('best3Offer') /1000)\
    .withColumn('lowest', col('lowest') /1000)\
    .withColumn('highest', col('highest') /1000)\
    .withColumn('refPrice', col('refPrice') /1000)\
    .withColumn('floor', col('floor') /1000)\
    .withColumn('ceiling', col('ceiling') /1000)\
    .withColumn('matchedPrice', col('matchedPrice') /1000)\
    .withColumn("time_stamp", to_timestamp(lit(time_stamp)))\
    .withColumnRenamed("exchange","stock_exchange")\
    .withColumnRenamed("stockSymbol","stock_code")

stock_info_clean = stock_info_clean.select(\
    col("time_stamp"),\
    col("stock_code"),\
    col("stock_exchange"),\
    col("best1Bid"),\
    col("best1BidVol"),\
    col("best2Bid"),\
    col("best2BidVol"),\
    col("best3Bid"),\
    col("best3BidVol"),\
    col("best1Offer"),\
    col("best1OfferVol"),\
    col("best2Offer"),\
    col("best2OfferVol"),\
    col("best3Offer"),\
    col("best3OfferVol"),\
    col("priceChange"),\
    col("priceChangePercent"),\
    col("nmTotalTradedQty"),\
    col("lowest"),\
    col("highest"),\
    col("matchedPrice"),\
    col("matchedVolume"),\
    col("refPrice"),\
    col("floor"),\
    col("ceiling"))

stock_info_clean.show(1, False)

# stock_info_clean.write.format("csv").option("header","true").mode("overwrite").save("hdfs://namenode:9000/project20221/clean/ssiDataApi.csv")
# stock_info1 = spark.read.option("header","true").csv("hdfs://namenode:9000/project20221/clean/ssiDataApi.csv")
# stock_info1.show(60, False)
# print(stock_info1.count())


+-------------------+----------+--------------+--------+-----------+--------+-----------+--------+-----------+----------+-------------+----------+-------------+----------+-------------+-----------+------------------+----------------+------+-------+------------+-------------+--------+-----+-------+
|time_stamp         |stock_code|stock_exchange|best1Bid|best1BidVol|best2Bid|best2BidVol|best3Bid|best3BidVol|best1Offer|best1OfferVol|best2Offer|best2OfferVol|best3Offer|best3OfferVol|priceChange|priceChangePercent|nmTotalTradedQty|lowest|highest|matchedPrice|matchedVolume|refPrice|floor|ceiling|
+-------------------+----------+--------------+--------+-----------+--------+-----------+--------+-----------+----------+-------------+----------+-------------+----------+-------------+-----------+------------------+----------------+------+-------+------------+-------------+--------+-----+-------+
|2023-01-10 21:45:01|MBS       |hnx           |14.1    |298300     |14.0    |206400     |13.9    |17540

In [96]:
all_stock_info = spark.read\
    .option("header",True)\
    .option("inferSchema", True)\
    .csv("hdfs://namenode:9000/project20221/clean/ssiDataApi.csv")
all_stock_info.printSchema()
all_stock_info.show(1, False)


root
 |-- time_stamp: timestamp (nullable = true)
 |-- stock_code: string (nullable = true)
 |-- stock_exchange: string (nullable = true)
 |-- best1Bid: double (nullable = true)
 |-- best1BidVol: integer (nullable = true)
 |-- best2Bid: double (nullable = true)
 |-- best2BidVol: integer (nullable = true)
 |-- best3Bid: double (nullable = true)
 |-- best3BidVol: integer (nullable = true)
 |-- best1Offer: double (nullable = true)
 |-- best1OfferVol: integer (nullable = true)
 |-- best2Offer: double (nullable = true)
 |-- best2OfferVol: integer (nullable = true)
 |-- best3Offer: double (nullable = true)
 |-- best3OfferVol: integer (nullable = true)
 |-- priceChange: double (nullable = true)
 |-- priceChangePercent: double (nullable = true)
 |-- nmTotalTradedQty: integer (nullable = true)
 |-- lowest: double (nullable = true)
 |-- highest: double (nullable = true)
 |-- matchedPrice: double (nullable = true)
 |-- matchedVolume: integer (nullable = true)
 |-- refPrice: double (nullable = tru

In [94]:
news = spark.read.json("hdfs://namenode:9000/project20221/raw/analysNews.json")

clean_news = news\
    .distinct()\
    .withColumn('time_stamp', to_timestamp(concat(col('newsDate'), lit(' '), col('newsTime'))))\
    .withColumnRenamed("tagCodes","stock_code")

clean_news = clean_news.select(\
    col("newsAbstract"),\
    col("newsContent"),\
    col("newsId"),\
    col("newsSource"),\
    col("newsTitle"),\
    col("newsUrl"),\
    col("stock_code"),\
    col("time_stamp"))


clean_news.write.format("json").mode("overwrite").save("hdfs://namenode:9000/project20221/clean/analysNews.json")

root
 |-- newsAbstract: string (nullable = true)
 |-- newsContent: string (nullable = true)
 |-- newsId: string (nullable = true)
 |-- newsSource: string (nullable = true)
 |-- newsTitle: string (nullable = true)
 |-- newsUrl: string (nullable = true)
 |-- stock_code: string (nullable = true)
 |-- time_stamp: timestamp (nullable = true)



In [3]:
# Analys news schema
analys_news_schema = StructType([\
    StructField("newsAbstract", StringType(), True),\
    StructField("newsContent", StringType(), True),\
    StructField("newsId", StringType(), True),\
    StructField("newsSource", StringType(), True),\
    StructField("newsTitle", StringType(), True),\
    StructField("newsUrl", StringType(), True),\
    StructField("stock_code", StringType(), True),\
    StructField("time_stamp", TimestampType(), True),\
])

news = spark\
    .read\
    .schema(analys_news_schema)\
    .json("hdfs://namenode:9000/project20221/clean/analysNews.json")
news.printSchema()
news.show(1, False)

NameError: name 'spark' is not defined

In [ ]:
fullInfo = companies.join(stockInfo, companies.stock_code == stockInfo.stockSymbol, "inner")
fullInfo.show(truncate=False)

In [ ]:
from pyspark.sql.functions import col

stock_code = 'BAF'
news.filter(col("tagCodes").like(stock_code)).show(1, False)

In [2]:
raw = spark.read.json("hdfs://namenode:9000/project20221/clean/stock_price_realtime/01-11-2023-22-28-35.json")




+--------+-----------+----------+-------------+--------+-----------+----------+-------------+--------+-----------+----------+-------------+-------+-----+-------+------+------------+-------------+----------------+-----------+------------------+--------+----------+--------------+--------------------+
|best1Bid|best1BidVol|best1Offer|best1OfferVol|best2Bid|best2BidVol|best2Offer|best2OfferVol|best3Bid|best3BidVol|best3Offer|best3OfferVol|ceiling|floor|highest|lowest|matchedPrice|matchedVolume|nmTotalTradedQty|priceChange|priceChangePercent|refPrice|stock_code|stock_exchange|          time_stamp|
+--------+-----------+----------+-------------+--------+-----------+----------+-------------+--------+-----------+----------+-------------+-------+-----+-------+------+------------+-------------+----------------+-----------+------------------+--------+----------+--------------+--------------------+
|    11.0|        100|      12.5|         1400|     0.0|          0|      12.6|         1000|     0.